In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import plotly.express as px

In [ ]:
data = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

In [ ]:
def preprocess_data(df):
    isDuplicate = df.duplicated().sum()
    Total = df.shape[0]
    return pd.DataFrame({'duplicates':[isDuplicate], 'total': [Total]})

In [ ]:
def hd(df, n):
    return df.head(n=n)

In [ ]:
def getNulls(df, order):
    return df.isnull().sum().sort_values(ascending = True if order is 'a' else False)

In [ ]:
preprocess_data(data)

In [ ]:
getNulls(data, 'd')

In [ ]:
print('The max duration is ', max(data.duration))
print('The min duration is ', min(data.duration))

In [ ]:
data.rating.unique()

In [ ]:
#rating
tv_ratings = {
   'TV-MA':'TV >17',
    'R': 'M >17',
    'PG-13': 'M >13',
    'TV-14': 'TV >14',
    'TV-PG': 'TV Parental Guidance',
    'NR':'Not Rated', 
    'TV-G': 'All', 
    'TV-Y': 'TV 2-6',
    'TV-Y7': 'TV >7', 
    'PG': 'Movie Parental Guidance',
    'G': 'General', 
    'NC-17': '>17', 
    'TV-Y7-FV': 'TV >7 FV', 
    'UR': 'Unrated'
}

In [ ]:
#data['ratings'] = [tv_ratings[i] for i in data.rating if i is not np.nan else 'Unrated']
data['ratings'] = ['Unrated' if i is np.nan else tv_ratings[i] for i in data.rating]

In [ ]:
data.groupby('type').count()

In [ ]:
px.box(data.release_year.sort_values())

In [ ]:
#Count of Number of Seasons per year

In [ ]:
d_type_yr = data.groupby(['type', 'release_year']).count().reset_index()

In [ ]:
px.bar(d_type_yr, x = 'release_year' ,y = 'show_id', color = 'type', barmode = 'group', range_x = [1990,2021], title = 'TV Shows Finally took over Movies in 2020')

In [ ]:
hd(data,5)

In [ ]:
import datetime

In [ ]:
data.loc[data.title == 'Friends', 'date_added'] = 'January 1, 2015'

In [ ]:
data = data[~data.date_added.isna()]

In [ ]:
data.date_added = [datetime.datetime.strptime(str(d).strip(), '%B %d, %Y') for d in data.date_added if d is not '']

In [ ]:
# In which year what was added and by how much

In [ ]:
data['year_add']    = [d.year for d in data.date_added]
data['month_add']   = [d.month for d in data.date_added]
data['day_add']     = [d.day for d in data.date_added]
data['weekday_add'] = [d.weekday() for d in data.date_added]

In [ ]:
data.head()

In [ ]:
# Year wise rise of content
px.bar(data.groupby(['type', 'year_add']).count().reset_index(), x = 'year_add', y = 'show_id', color = 'type', barmode = 'group',  title = 'Steady Increase in TV Show')

In [ ]:
px.bar(data.groupby(['month_add']).count().reset_index(), x = 'month_add', y = 'show_id')

In [ ]:
hd(data,5)

In [ ]:
import re

In [ ]:
data_with_countries = data[~data.country.isnull()]
data_with_countries['one_country'] = [len(re.findall(',',i)) for i in data_with_countries.country]
dt_one_cntry = data_with_countries[data_with_countries.one_country == 0]

In [ ]:
shows_count_by_country = dt_one_cntry.groupby(['country', 'type']).count().reset_index().sort_values(by='show_id', ascending = False)
top_20_countries_only_tv_shows = shows_count_by_country[shows_count_by_country.type == 'TV Show'].head(n=20)
top_20_countries_only_movies = shows_count_by_country[shows_count_by_country.type == 'Movie'].head(n=20)

In [ ]:
px.bar(top_20_countries_only_movies, x = 'country', y = 'show_id', title = 'Movies')

In [ ]:
px.bar(top_20_countries_only_tv_shows, x = 'country', y = 'show_id', title = 'TV Shows')

In [ ]:
data['num_duration'] = [int(re.findall('\d+',i)[0]) for i in data.duration]

In [ ]:
print('Minimum')
print(data.groupby('type')['num_duration'].min())
print('--------------------------------------------')
print('Maximum')
print(data.groupby('type')['num_duration'].max())

In [ ]:
data[((data.num_duration == 3) & (data.type == 'Movie'))]

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.corpus import stopwords

In [ ]:
#How many animated movies

In [ ]:
data['desc_token'] = [word_tokenize(w) for w in data.description]

In [ ]:
data['desc_token_no_stop'] = [[w for w in pert if w not in stopwords.words('english')] for pert in data.desc_token]


In [ ]:
wordnetLem = WordNetLemmatizer()

In [ ]:
data['lem_tokens'] = [[wordnetLem.lemmatize(w) for w in per] for per in data.desc_token_no_stop]

In [ ]:
data['isanimated'] = [sum([w.lower() in ['animate','animated'] for w in per]) for per in data.lem_tokens]

In [ ]:
data[data.isanimated > 0].head()

In [ ]:
# Count of Shows by directors

In [ ]:
der_type = data[['type','director']].dropna()

In [ ]:
der_type.groupby('director').count().reset_index().sort_values(by = 'type', ascending = False)

In [ ]:
der_type['director_ind'] = [d.split(',') for d in der_type.director]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectCV = CountVectorizer(ngram_range=(2,2)) 

In [ ]:
val_mat = vectCV.fit_transform(der_type.director)

In [ ]:
feat_name = vectCV.get_feature_names()

In [ ]:
df_count = pd.DataFrame(val_mat.toarray(), columns = feat_name)

In [ ]:
df_count.sum(axis = 0).sort_values(ascending = False)

In [ ]:
der_type['boll'] = ['Raúl Campos' in i for i in der_type['director_ind']]

In [ ]:
der_type['boll2'] = [len(re.findall('Raúl Campos', i)) for i in der_type['director']]

In [ ]:
data[data.director == 'Jan Suter, Raúl Campos Delgado']

In [ ]:
#Audience Rating wise - General Ratings wise count; Highest rating among country; trend over the years

In [ ]:
px.line(data.groupby(['ratings', 'year_add']).count().reset_index(), x = 'year_add', y = 'show_id', color = 'ratings')

In [ ]:
px.line(data.groupby(['type','ratings', 'year_add']).count().reset_index(), x = 'year_add', y = 'show_id', color = 'ratings', facet_col = 'type')

In [ ]:
data_only_tv_shows = data[data.type == 'TV Show']

In [ ]:
data_only_tv_shows.head()

In [ ]:
px.histogram(data_only_tv_shows, x = 'num_duration')

In [ ]:
dt_tv_shows_top_three = data_only_tv_shows[((data_only_tv_shows.country == 'India') | (data_only_tv_shows.country == 'United States') | (data_only_tv_shows.country == 'United Kingdom'))]

In [ ]:
px.histogram(dt_tv_shows_top_three, x = 'num_duration', color = 'country', barmode = 'overlay')

In [ ]:
data_only_movies = data[data.type == 'Movie']

In [ ]:
data_only_movies.head()

In [ ]:
px.histogram(data_only_movies, x = 'num_duration', marginal = 'box', labels = {'num_duration':'Duration(in mins)'})

In [ ]:
#check for top country compared with India
dt_top_two_country_movie = data_only_movies[((data_only_movies.country == 'India') | (data_only_movies.country == 'United States'))]

In [ ]:
px.histogram(dt_top_two_country_movie, x = 'num_duration', color = 'country', histnorm = 'percent', marginal = 'box', barmode = 'overlay')

In [ ]:
ratings_countr = dt_top_two_country_movie.groupby(['country','ratings']).count().reset_index()

In [ ]:
px.bar(ratings_countr, x = 'ratings', y = 'show_id', color = 'country', barmode = 'overlay', log_y = True)

In [ ]:
dt_with_directors = data[~data.director.isnull()]

In [ ]:
dt_cast_direct = dt_with_directors[['director','cast']]

In [ ]:
dt_cast_direct = dt_cast_direct[~dt_cast_direct.cast.isnull()]

In [ ]:
dt_cast_direct.info()

In [ ]:
dt_cast_direct['cast_split'] = [re.split(',', i) for i in  dt_cast_direct.cast]

In [ ]:
dt_cast_direct.drop(columns = 'cast', inplace = True)

In [ ]:
dt_cast_direct_test = dt_cast_direct.head()

In [ ]:
dt_cast_direct['pairs'] = [[dt_cast_direct.director.iloc[index]+'_'+ i.strip() for i in c] for index,c in enumerate(dt_cast_direct.cast_split)]

In [ ]:
#dt_cast_direct['pairs']
final_list = []
for i in dt_cast_direct['pairs']:
    final_list.append(i)

In [ ]:
newlist = sum(final_list, []) #flatten the list

In [ ]:
dft = pd.DataFrame({'pairs':newlist})

In [ ]:
dft.pairs.value_counts()[1:50]

In [ ]:
data[data.director == 'Yılmaz Erdoğan']

In [ ]:
data['listed_in'] = [i.split(',') for i in data.listed_in]

In [ ]:
data_exploded = data.explode('listed_in')

In [ ]:
data_exp_grpd_cnt = data_exploded.groupby(['country','type','listed_in']).count().sort_values(by = 'show_id', ascending = False).reset_index()[['country','type','listed_in','show_id']]
data_exp_grpd_cnt.rename(columns = {'show_id':'count_of'}, inplace = True)
data_exp_grpd_cnt.sort_values(by = ['country','count_of'], ascending = [True,False],inplace = True)
data_exp_grpd_cnt = data_exp_grpd_cnt.reset_index()
data_exp_grpd_cnt.drop(columns = 'index', inplace = True)

In [ ]:
data_exp_grpd_cnt['percent'] = ([round(data_exp_grpd_cnt.loc[index,'count_of']/sum(data_exp_grpd_cnt[data_exp_grpd_cnt.country == i]['count_of']),4)*100 for index,i in enumerate(data_exp_grpd_cnt.country)])

In [ ]:
top_3_by_genre = data_exp_grpd_cnt.groupby('country').head(3)

In [ ]:
data_exp_grpd_cnt